#### Notes

1.   If we are providing a solution for link prediction, we might consider using imbalanced data with more negative samples to mimic the real world scenario. 
2.   We can think of a better approach than randomly selecting edges and checking if they are +ve/-ve. We can do this by factoring in a weight related to the path between two nodes.

#Imports + Installations

In [1]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans # Clustering
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split
import collections
from scipy.sparse.linalg import eigs

#Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
os.listdir()

['.config', 'gdrive', 'sample_data']

In [4]:
os.listdir('gdrive/My Drive/Major Project/')

['soc-Epinions1.txt',
 'pos_samples_test_rand.csv',
 'neg_samples.csv',
 'shuffled_df.csv',
 'df_train.csv',
 'pos_samples.csv',
 'train_data_gw',
 'test_data_gw',
 'Major Projectgraph_wave_embeddings',
 'neg_test.csv',
 'neg_train',
 'combined.csv',
 'combined_test.csv',
 'nodesketch_pkl',
 'train_data_nodesketch_concat',
 'test_data_nodesketch_concat',
 'graphsage_model.h5',
 'wys_model.h5',
 'wys_file.pkl',
 'wys_embeddings.pkl',
 'train_91.csv',
 'test_91.csv',
 'pos_train_91.csv',
 'nodesketch91_pkl',
 'train_data_nodesketch91',
 'test_data_nodesketch91',
 'fea_sample',
 'wys_embeddings_91.pkl',
 'wys_model_91.h5',
 'wiki']

In [5]:
data_path = 'gdrive/My Drive/Major Project/wiki/'

In [6]:
train = pd.read_csv(data_path+'/Wiki-Vote.txt', sep = "\t", header = None, names = ['Source', 'Destination'])
train.head()

,Source,Destination
0,30,1412
1,30,3352
2,30,5254
3,30,5543
4,30,7478


In [7]:
nodes = sorted(list(set(train['Source'].values) | set (train['Destination'].values)))
print(len(nodes))
ledger = {}
for i, node in enumerate(nodes):
  ledger[node] = i
print(len(ledger))

7115
7115


In [8]:
graph = nx.DiGraph()
for src, dst in tqdm(train.values):
  graph.add_edge(ledger[src], ledger[dst])

  0%|          | 0/103689 [00:00<?, ?it/s]

In [9]:
df = pd.DataFrame({}, {}, columns = ['src', 'dest'])  
src_list = [ledger[src] for src,dest in train.values]
dest_list = [ledger[dest] for src, dest in train.values]
df['src'] = src_list
df['dest'] = dest_list
df['link'] = 1
print(df.head())

   src  dest  link
0   27  1299     1
1   27  3064     1
2   27  4680     1
3   27  4919     1
4   27  6484     1


In [10]:
df = df.sample(frac=1, random_state=25)

In [11]:
node1 = set(df['src'])
node2 = set(df['dest'])
df_nodes = node1.union(node2)
print(len(df_nodes))

7115


In [12]:
graph.number_of_nodes()

7115

In [13]:
graph.number_of_edges()

103689

In [14]:
for node in graph.nodes():
  if (len(graph.out_edges(node)) + len(graph.in_edges(node))) == 0:
    print(node)

in_out_zero = []
for node in graph.nodes():
  if len(graph.out_edges(node)) == 0 or len(graph.in_edges(node)) == 0:
    in_out_zero.append(node)

len(in_out_zero)

5739

In [15]:
nodelist = sorted(list(ledger.values()))
nodes = sorted(list(graph.nodes()))
print(len(nodelist))
if nodes == nodelist:
  print("true")
else:
  print("false")

7115
true


In [16]:
graph.adj[3]

AtlasView({0: {}, 5: {}, 7: {}, 16: {}, 20: {}, 25: {}, 26: {}, 27: {}, 30: {}, 31: {}, 32: {}, 35: {}, 36: {}, 47: {}, 51: {}, 52: {}, 53: {}, 58: {}, 75: {}, 81: {}, 84: {}, 88: {}, 89: {}, 121: {}, 124: {}, 125: {}, 128: {}, 133: {}, 140: {}, 143: {}, 145: {}, 146: {}, 155: {}, 159: {}, 160: {}, 162: {}, 166: {}, 169: {}, 173: {}, 174: {}, 205: {}, 208: {}, 214: {}, 215: {}, 216: {}, 219: {}, 223: {}, 230: {}, 237: {}, 239: {}, 244: {}, 245: {}, 246: {}, 248: {}, 257: {}, 258: {}, 259: {}, 263: {}, 267: {}, 268: {}, 272: {}, 280: {}, 281: {}, 282: {}, 283: {}, 286: {}, 288: {}, 299: {}, 301: {}, 304: {}, 307: {}, 321: {}, 330: {}, 331: {}, 337: {}, 339: {}, 340: {}, 341: {}, 342: {}, 349: {}, 350: {}, 352: {}, 356: {}, 357: {}, 358: {}, 367: {}, 368: {}, 369: {}, 371: {}, 373: {}, 379: {}, 382: {}, 383: {}, 384: {}, 392: {}, 397: {}, 398: {}, 403: {}, 404: {}, 407: {}, 408: {}, 409: {}, 411: {}, 415: {}, 416: {}, 422: {}, 486: {}, 493: {}, 497: {}, 508: {}, 512: {}, 514: {}, 515: {}

# Section 1: Data Preparation

1.1 Extracting positive samples from the graph

The procedure followed is:
* Obtain the adjacency matrix
* Traverse to find nodes that are not connected
* Store these pairs of nodes to use as negative samples during training

In [17]:
list(graph.edges())[:5]

[(27, 1299), (27, 3064), (27, 4680), (27, 4919), (27, 6484)]

In [20]:
def get_neg_samples(nodes, graph):
  """
  Input: 
    nodes: list of all nodes of the graph
    graph: the nx graph
  Output:
    neg_samples: a pd dataframe containing all pairs of disconnected nodes
  
  Computes the adjacency matrix, and returns all disconnected nodes
  """

  edges = set(graph.edges())

  discon_pairs = set([])

  while (len(discon_pairs)<104000):
    try:
      a=ledger[random.randint(1, 7115)]
      b=ledger[random.randint(1, 7115)]
    except:
      continue
    if a!=b and (a,b) not in edges and (b,a) not in edges:
        try:
            if nx.shortest_path_length(graph,source=a,target=b) > 2: 
                discon_pairs.add((a,b))
            else:
                continue  
        except:  
                discon_pairs.add((a,b))              
    else:
        continue

  # for i in tqdm(range(adj_g.shape[0]-1)):
  #   for j in range(i+1, adj_g.shape[1]):
  #     try:
  #       if nx.shortest_path_length(graph, i, j) <= 2:
  #         if adj_g[i, j] == 0:
  #           discon_pairs.append([nodes[i], nodes[j]])
  #     except nx.NetworkXNoPath:
  #       continue

  print(f"Number of negative links found = {len(discon_pairs)}")

  #converting the list into a dataframe

  neg_samples = pd.DataFrame({'src': [i[0] for i in discon_pairs], 
                              'dest': [i[1] for i in discon_pairs]})  
  neg_samples['link'] = 0

  return neg_samples

In [21]:
%%time
neg_data = get_neg_samples(nodelist, graph)

# estimated running time for this cell = 4 days
# check for possible places for optimization 

Number of negative links found = 104000
CPU times: user 2.7 s, sys: 271 µs, total: 2.7 s
Wall time: 2.7 s


In [22]:
neg_data.head()

,src,dest,link
0,2202,2273,0
1,5852,400,0
2,6163,3526,0
3,2121,2551,0
4,824,2104,0


In [23]:
neg_data.shape

(104000, 3)

1.2 Obtain positive samples from the original graph

These positive samples, along with the negative samples obtained above would be used for training the classifiers. The positive samples are simply **pairs of nodes between which a link actually exists in the original graph.**

The procedure followed is: 
* Check if dropping a node pair results in splitting of the graph (increases the number of conn components)
* Check if dropping the pair results in a reduction in the number of nodes
* If both the above constraints are fulfilled, then drop the node pair (edge) - store this node pair in a list of positive samples

In [24]:
# creating a dictionary of number of times a node occurs in the dict
src = list(df['src'])
dest = list(df['dest'])

src_dict = collections.Counter(src)
dest_dict = collections.Counter(dest)

print(len(src_dict), len(dest_dict))

6110 2381


In [25]:
def get_pos_samples(df, g):
  """
  Input:
    df: the original edgelist as a pd dataframe 
    g: the original networkx graph
    assumes: 
      nodes are specified as Source and Destination
      original Graph is a Digraph
      node indexing is corrected
  Output:
    pos_samples: dataframe containing all removable edges (positive samples)
  """
  df_temp = df.copy()
  gr = g.copy(as_view=False)
  node_count = len(gr.nodes())
  print(f"original node count = {node_count}")
  
  removable_links_idx = []
  
  cc = nx.number_weakly_connected_components(gr)
  print(f"Original number of connected components = {cc}")

  for c, i in tqdm(enumerate(df.index.values)):
    src, dest, link = df.loc[i].values
    
    gr.remove_edge(src,dest)

    if(gr.degree(src) == 0):
      gr.remove_node(src)
    if(gr.degree(dest) == 0):
      gr.remove_node(dest)
  
    if len(gr.nodes()) == node_count:
      removable_links_idx.append(i)
      df_temp = df_temp.drop(index=i)
    else:
      gr.add_edge(src,dest)
    
    if c == 10500:
      print(f"removable links index = {len(removable_links_idx)}")
      break
  
  pos_samples = df.loc[removable_links_idx]
  pos_samples['link'] = 1

  nodes_new = set(df_temp['src']).union(set(df_temp['dest']))
  print(f"New number of nodes = {len(nodes_new)}")

  return pos_samples, df_temp

In [26]:
pos_samples, df_train = get_pos_samples(df, graph)
pos_samples_idx = pos_samples.index.values
print(pos_samples.shape)
pos_samples.head()

original node count = 7115
Original number of connected components = 24


0it [00:00, ?it/s]

removable links index = 10267
New number of nodes = 7115
(10267, 3)


,src,dest,link
11062,295,1471,1
30200,1183,2760,1
102674,6672,6977,1
65110,3146,4845,1
63022,3056,2774,1


In [27]:
print(df_train.shape)
df_train.head()

(93422, 3)


,src,dest,link
33079,1332,1211,1
100635,6224,3649,1
61866,2909,2439,1
86957,4753,2163,1
103670,7082,5806,1


In [28]:
# sanity check
train_nodes = set(df_train['src']).union(set(df_train['dest']))

In [29]:
len(df_nodes.difference(train_nodes))

0

In [30]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph())

In [31]:
original_graph = nx.from_pandas_edgelist(df, source="src", target="dest", create_using=nx.MultiDiGraph)

In [32]:
cc_before = nx.number_weakly_connected_components(original_graph)
print(cc_before)

24


In [33]:
cc_after = nx.number_weakly_connected_components(train_graph)

In [34]:
print(cc_after)

24


In [35]:
neg_data.shape

(104000, 3)

In [36]:
neg_test = neg_data.sample(frac = 0.1)
print(neg_test.shape)
neg_test.head()

(10400, 3)


,src,dest,link
80658,3023,2594,0
94884,354,3661,0
27031,2751,6070,0
46290,3549,1901,0
2102,3397,3564,0


In [37]:
neg_train = neg_data.drop(neg_test.index.values, axis=0)
print(neg_train.shape)
neg_train.head()

(93600, 3)


,src,dest,link
0,2202,2273,0
1,5852,400,0
2,6163,3526,0
3,2121,2551,0
4,824,2104,0


In [38]:
combined_tr = pd.concat([df_train, neg_train], axis = 0).sample(frac=1)
print(combined_tr.shape)
combined_tr.head()

(187022, 3)


,src,dest,link
11538,308,75,1
2338,3874,251,0
89956,1506,4335,0
54348,2471,4066,1
91913,1584,2951,0


In [39]:
combined_te = pd.concat([pos_samples, neg_test], axis = 0).sample(frac=1)
print(combined_te.shape)
combined_te.head()

(20667, 3)


,src,dest,link
13017,415,81,1
98801,2230,1590,0
4421,67,4680,1
57102,2633,2303,1
62449,2999,698,1


In [40]:
combined_tr.to_csv(data_path+'/train_91.csv')
combined_te.to_csv(data_path+'/test_91.csv')

In [41]:
df_train.to_csv(data_path+'/pos_train_91.csv')

In [42]:
combined_tr = pd.read_csv(data_path+'/train_91.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(187022, 3)


,src,dest,link
11538,308,75,1
2338,3874,251,0
89956,1506,4335,0
54348,2471,4066,1
91913,1584,2951,0


In [43]:
combined_te = pd.read_csv(data_path+'/test_91.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(20667, 3)


,src,dest,link
13017,415,81,1
98801,2230,1590,0
4421,67,4680,1
57102,2633,2303,1
62449,2999,698,1


In [44]:
df_train = pd.read_csv(data_path+'/pos_train_91.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(93422, 3)


,src,dest,link
33079,1332,1211,1
100635,6224,3649,1
61866,2909,2439,1
86957,4753,2163,1
103670,7082,5806,1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph)

In [ ]:
scales = [5, 10, 20, 50]
sample_points = np.linspace(0, 100, 50).astype(np.float32)
degree = 20
G = StellarGraph.from_networkx(train_graph)
generator = GraphWaveGenerator(G, scales=scales, degree=degree)

In [ ]:
emb = generator.flow(
    node_ids = G.nodes(), sample_points = sample_points, batch_size = 1, repeat = False
)

In [ ]:
embeddings = [x.numpy() for x in tqdm(emb)]

  0%|          | 0/75879 [00:00<?, ?it/s]

In [ ]:
filename = data_path + "_embeddings"

In [ ]:
outfile = open(filename, "wb")
pickle.dump(embeddings, outfile)

In [ ]:
print(filename)

gdrive/My Drive/Major Project_embeddings


In [ ]:
file = open(filename, "rb")
gw_emb = pickle.load(file)
file.close()

EOFError: ignored

In [ ]:
print(len(gw_emb))

75879


In [ ]:
emb_dict = {}
for node, emb in zip(G.nodes(), gw_emb):
  emb_dict[node] = emb
print(len(emb_dict))

75879


In [ ]:
# get training embeddings
t_e = []
for i, row in combined.iterrows():
  comb_emb = emb_dict[row['src']] + emb_dict[row['dest']]
  t_e.append(comb_emb)
print(len(t_e))

999165


In [ ]:
t_y = combined['link']
len(t_y)

999165

In [ ]:

import xgboost as xgb
